Check for authentication_credentials.csv

In [0]:
dbutils.fs.ls("/FileStore/tables")

Out[13]: [FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv', name='authentication_credentials.csv', size=202, modificationTime=1687110999000),
 FileInfo(path='dbfs:/FileStore/tables/geo_dirty.csv/', name='geo_dirty.csv/', size=0, modificationTime=1700734886599),
 FileInfo(path='dbfs:/FileStore/tables/pin_dirty.csv/', name='pin_dirty.csv/', size=0, modificationTime=1700734886599),
 FileInfo(path='dbfs:/FileStore/tables/user_dirty.csv/', name='user_dirty.csv/', size=0, modificationTime=1700734886599)]

In [0]:
import urllib

Read the csv file containing the AWS keys to Databricks

In [0]:
aws_keys_df =  (spark.read.format("csv")
                .option("header", "true")
                .option("sep", ",")
                .load("/FileStore/tables/authentication_credentials.csv")
)

Extract AWS access key and secret key from spark dataframe

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

Mount the S3 bucket

In [0]:
AWS_S3_BUCKET = "user-0e36c8cd403d-bucket"
MOUNT_NAME = "/mnt/0e36c8cd403d_s3_bucket"
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)

# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[12]: True